# Knowledge Retrieval

https://platform.openai.com/docs/assistants/tools/knowledge-retrieval

In [2]:
from openai import OpenAI
import os, toml
secrets = toml.load("../.streamlit/secrets.toml")
os.environ['OPENAI_API_KEY'] = secrets['OPEN_AI_KEY']
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
)

If you enable retrieval for a specific Assistant, all the files attached will be automatically indexed and you will be charged the $0.20/GB per assistant per day. You can enabled/disable retrieval by using the Modify Assistant endpoint.

In [6]:
import requests

url = "https://www.mathworks.com/help/pdf_doc/matlab/learn_matlab.pdf"
response = requests.get(url)
binary_data = response.content

# You can now use the binary data as needed
# For example, you can save it to a file
# with open("data/learn_matlab.pdf", "wb") as file:
#     file.write(binary_data)


In [7]:
# Upload a file with an "assistants" purpose
file = client.files.create(
  file=binary_data,
  purpose='assistants'
)
file

FileObject(id='file-9SceeVNRanZno0c3TzGwSwtj', bytes=2012002, created_at=1705090985, filename='upload', object='file', purpose='assistants', status='processed', status_details=None)

In [8]:
# Add the file to the assistant
assistant = client.beta.assistants.create(
  name="MATLAB Primer",
  instructions="You are a MATLAB Assistant. Use your knowledge base to best respond to customer queries.",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)
assistant

Assistant(id='asst_lsvO9XkKXRoPnOqglWWVQJ8q', created_at=1705091060, description=None, file_ids=['file-9SceeVNRanZno0c3TzGwSwtj'], instructions='You are a MATLAB Assistant. Use your knowledge base to best respond to customer queries.', metadata={}, model='gpt-4-1106-preview', name='MATLAB Primer', object='assistant', tools=[ToolRetrieval(type='retrieval')])

In [9]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "how to write text in string arrays",
      "file_ids": [file.id]
    }
  ]
)

In [10]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # instructions="Please address the user as Jane Doe. The user has a premium account."
    )
run

Run(id='run_KSVHvi78aaZfJvDiYQXQVCHT', assistant_id='asst_lsvO9XkKXRoPnOqglWWVQJ8q', cancelled_at=None, completed_at=None, created_at=1705091672, expires_at=1705092272, failed_at=None, file_ids=['file-9SceeVNRanZno0c3TzGwSwtj'], instructions='You are a MATLAB Assistant. Use your knowledge base to best respond to customer queries.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_VURvtaDMGSDqU8EHXPJx81fn', tools=[ToolAssistantToolsRetrieval(type='retrieval')])

In [11]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
for m in messages.data:
    print(m.role)
    print(m.content[0].text.value)
    print('--')

assistant
In MATLAB, you can write text into string arrays using either double quotes (`" "`) for creating string scalars, or curly braces (`{ }`) when you want to manually create a cell array of character vectors. Below are examples of how you can do this:

```matlab
% Using double quotes to create a string array (recommended)
strArray = ["hello", "world", "this", "is", "a", "string", "array"];

% Using single quotes to create a cell array of character vectors
cellArrayOfStrings = {'hello', 'world', 'this', 'is', 'a', 'string', 'array'};

% Adding a new text into an existing string array
strArray = [strArray, "new text"];

% If you have a cell array of character vectors, you can add another text like this:
cellArrayOfStrings = [cellArrayOfStrings, {'another text'}];
```

String arrays are more versatile and generally recommended over cell arrays for text data since MATLAB introduced them in version R2016b. They support most operations that you would typically use with character arrays

In [16]:
print(messages.dict().keys())

dict_keys(['data', 'object', 'first_id', 'last_id', 'has_more'])


In [22]:
print(messages.data[0].content[0].text.value)

In MATLAB, you can write text into string arrays using either double quotes (`" "`) for creating string scalars, or curly braces (`{ }`) when you want to manually create a cell array of character vectors. Below are examples of how you can do this:

```matlab
% Using double quotes to create a string array (recommended)
strArray = ["hello", "world", "this", "is", "a", "string", "array"];

% Using single quotes to create a cell array of character vectors
cellArrayOfStrings = {'hello', 'world', 'this', 'is', 'a', 'string', 'array'};

% Adding a new text into an existing string array
strArray = [strArray, "new text"];

% If you have a cell array of character vectors, you can add another text like this:
cellArrayOfStrings = [cellArrayOfStrings, {'another text'}];
```

String arrays are more versatile and generally recommended over cell arrays for text data since MATLAB introduced them in version R2016b. They support most operations that you would typically use with character arrays and provi